### ALG

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

nodes = gpd.read_file(r'E:/v423_ONTW_SPM/04 Diverse onderzoeken/34 Gesloten wegvakprobleem/Masternetw/MASTER_2022_20240808_ACT_node.SHP')
links = gpd.read_file(r'E:/v423_ONTW_SPM/04 Diverse onderzoeken/34 Gesloten wegvakprobleem/Masternetw/MASTER_2022_20240808_ACT_link.SHP')

In [2]:
export = False

In [3]:
#links[['NO', 'FROMNODENO', 'TONODENO']] = links[['NO', 'FROMNODENO', 'TONODENO']].astype('object')
#links.insert(len(links.columns) - 1, 'TYPE', '')
links

,NO,FROMNODENO,TONODENO,TYPENO,TSYSSET,NUMLANES,VISUEELL~1,V_WET,geometry
0,2,302,805,11,"BUS,FIETS,PW,VR",4,4,NaN,"LINESTRING (656084.017 693089.769, 656066.497 ..."
1,4,450,308,11,"BUS,FIETS,PW,VR",1,4,NaN,"LINESTRING (656906.921 688040.541, 656888.69 6..."
2,7,664,1458,11,"BUS,FIETS,PW,VR",1,4,NaN,"LINESTRING (647745.345 694555.199, 647762.172 ..."
3,7,1458,664,11,"BUS,FIETS,PW,VR",2,4,NaN,"LINESTRING (647884.867 694609.67, 647860.328 6..."
4,8,795,792,11,"BUS,FIETS,PW,VR",1,4,NaN,"LINESTRING (655690.114 692357.27, 655726.659 6..."
...,...,...,...,...,...,...,...,...,...
14065,119702,92789,93452,1,"BUS,FIETS,PW,VR",2,1,NaN,"LINESTRING (673985.783 714923.558, 674074.719 ..."
14066,119706,91852,93456,1,"BUS,FIETS,PW,VR",2,1,NaN,"LINESTRING (669422.401 709197.621, 669511.291 ..."
14067,119722,93473,92381,1,"BUS,FIETS,PW,TRA,VR",2,1,NaN,"LINESTRING (645055.797 747352.979, 645066.748 ..."
14068,119723,92382,85088,1,"BUS,FIETS,PW,TRA,VR",2,1,NaN,"LINESTRING (645874.761 744758.113, 645886.477 ..."


In [4]:
links['VISUEELL~1'].value_counts()

VISUEELL~1
4    6296
1    5852
3     961
2     961
Name: count, dtype: int64

In [5]:
nodes = nodes[['NO', 'CODE', 'L_IN_NO', 'L_IN_F_NO', 'L_IN_T_NO', 'L_IN_VIS', 'L_IN_VWET',
       'L_OUT_NO', 'L_OUT_F_NO', 'L_OUT_T_NO', 'L_OUT_VIS', 'L_OUT_VWET', 'STUDIEGE~3']]

nodes = nodes[nodes['STUDIEGE~3'] <= 2]
nodes

,NO,CODE,L_IN_NO,L_IN_F_NO,L_IN_T_NO,L_IN_VIS,L_IN_VWET,L_OUT_NO,L_OUT_F_NO,L_OUT_T_NO,L_OUT_VIS,L_OUT_VWET,STUDIEGE~3
0,4,None,91,674,4,4,None,None,None,None,None,None,1.0
1,5,None,None,None,None,None,None,78,5,1430,4,None,1.0
2,6,None,2562,74,6,4,None,2047,6,810,4,None,1.0
3,9,None,446,358,9,4,None,796,9,360,4,None,1.0
4,39,None,"1943,2509","1210,1452",39,4,None,"1943,2005",39,"1210,1454",4,None,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11023,93268,None,119429,86987,93268,4,None,119430,93268,89540,4,None,1.0
11024,93274,None,"119442,119443","89634,91735",93274,4,None,"119442,119443",93274,"89634,91735",4,None,1.0
11025,93298,None,"116684,119487","88803,88850",93298,4,None,119493,93298,93300,4,None,1.0
11026,93299,None,119488,88830,93299,4,None,"119489,119490",93299,"88851,91553",4,None,1.0


In [6]:
nodes.dtypes

NO              int32
CODE           object
L_IN_NO        object
L_IN_F_NO      object
L_IN_T_NO      object
L_IN_VIS       object
L_IN_VWET      object
L_OUT_NO       object
L_OUT_F_NO     object
L_OUT_T_NO     object
L_OUT_VIS      object
L_OUT_VWET     object
STUDIEGE~3    float64
dtype: object

In [7]:
links.dtypes

NO               int32
FROMNODENO       int32
TONODENO         int32
TYPENO          object
TSYSSET         object
NUMLANES         int32
VISUEELL~1       int32
V_WET          float64
geometry      geometry
dtype: object

### 1e opsplitsing punten obv van # ink en uitgaande links

In [8]:
#knopen met 1 uitgaande & 1 inkomende link
nodes_1in_1out = nodes[
    ~(nodes[['L_IN_NO', 'L_OUT_NO']].apply(lambda x: x.str.contains(',', na=False)).any(axis=1))
]

#knopen met meer dan 1 uitgaande of inkomende link
nodes_multipe_in_or_out = nodes[
    (nodes[['L_IN_NO', 'L_OUT_NO']].apply(lambda x: x.str.contains(',', na=False)).any(axis=1))
]

nodes_1in_1out

,NO,CODE,L_IN_NO,L_IN_F_NO,L_IN_T_NO,L_IN_VIS,L_IN_VWET,L_OUT_NO,L_OUT_F_NO,L_OUT_T_NO,L_OUT_VIS,L_OUT_VWET,STUDIEGE~3
0,4,None,91,674,4,4,None,None,None,None,None,None,1.0
1,5,None,None,None,None,None,None,78,5,1430,4,None,1.0
2,6,None,2562,74,6,4,None,2047,6,810,4,None,1.0
3,9,None,446,358,9,4,None,796,9,360,4,None,1.0
5,42,None,968,218,42,4,None,None,None,None,None,None,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11018,93251,None,None,None,None,None,None,119398,93251,93253,4,None,1.0
11019,93253,None,119398,93251,93253,4,None,119399,93253,85250,4,None,1.0
11021,93257,None,119406,85376,93257,2,None,119407,93257,87780,2,None,1.0
11022,93258,None,119408,85503,93258,1,None,119409,93258,85479,1,None,1.0


### 1e selectie snelwegen uit knopen met 1 ink & 1 uitg link

In [9]:
def selectie_knopen_obv_vistype(df_knoop, vis_in=[], vis_out=[]):
    #snelwegen en ringwegen KNOPEN selecteren obv visueellinktype ['1', '2']
    #op- en afrit KNOPEN selecteren obv visueellinktype ['3', '4']
    selectie_knopen_obv_vis = df_knoop[
        df_knoop['L_IN_VIS'].isin(vis_in) &
        (df_knoop['L_OUT_VIS'].isin(vis_out) | (df_knoop['L_OUT_VIS'] == ''))
    ]
    return selectie_knopen_obv_vis
    #nodes_1in_1out.query("L_IN_VIS in ['1', '2'] and L_OUT_VIS in ['1', '2']") #doet hetzelfde
    # lege eindes ook toevoegen

In [10]:
eerste_selectie_knopen_snelwegen = selectie_knopen_obv_vistype(nodes_1in_1out, vis_in=['1', '2'], vis_out=['1', '2'])
eerste_selectie_knopen_snelwegen

,NO,CODE,L_IN_NO,L_IN_F_NO,L_IN_T_NO,L_IN_VIS,L_IN_VWET,L_OUT_NO,L_OUT_F_NO,L_OUT_T_NO,L_OUT_VIS,L_OUT_VWET,STUDIEGE~3
62,1455,None,1010,1454,1455,1,None,2549,1455,1460,1,None,1.0
64,1457,None,1001,1459,1457,1,None,2548,1457,1453,1,None,1.0
68,1461,None,2552,1460,1461,1,None,1006,1461,1463,1,None,1.0
69,1462,None,1002,1465,1462,1,None,2551,1462,1459,1,None,1.0
70,1463,None,1006,1461,1463,1,None,1005,1463,1464,1,None,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11002,92845,None,115781,89866,92845,1,None,115780,92845,89869,1,None,1.0
11011,93195,None,119288,93194,93195,1,None,119287,93195,85411,1,None,1.0
11014,93199,None,119295,85481,93199,1,None,119296,93199,93198,1,None,1.0
11021,93257,None,119406,85376,93257,2,None,119407,93257,87780,2,None,1.0


In [11]:
def filter_links_obv_knopen(df_knoop, df_links, link_type_var='', link_type_value=''):
    print(df_knoop.duplicated(['L_IN_NO', 'L_IN_F_NO', 'L_IN_T_NO']).sum() == 0)
    print(df_knoop.duplicated(['L_OUT_NO', 'L_IN_F_NO', 'L_IN_T_NO']).sum() == 0)

    #Links filteren obv knopen eerste_selectie_snelwegen
    # Create sets of tuples from eerste_selectie_snelwegen
    in_tuples = set(df_knoop[['L_IN_NO', 'L_IN_F_NO', 'L_IN_T_NO']].itertuples(index=False, name=None))
    out_tuples = set(df_knoop[['L_OUT_NO', 'L_OUT_F_NO', 'L_OUT_T_NO']].itertuples(index=False, name=None))
    print(len(in_tuples), len(out_tuples))
    
    # Combine both sets tuples
    valid_tuples = in_tuples.union(out_tuples)
    valid_tuples_int = set(tuple(map(np.int32, t)) for t in valid_tuples)
    print(len(valid_tuples_int))
    
    # Filter the links df
    df_links_selectie = df_links[df_links[['NO', 'FROMNODENO', 'TONODENO']].apply(tuple, axis=1).isin(valid_tuples_int)]
    df_links_selectie[link_type_var] = link_type_value

    return df_links_selectie

In [12]:
links_hoofdrijbaan_1 = filter_links_obv_knopen(eerste_selectie_knopen_snelwegen, links, link_type_var='TYPE_1', link_type_value='hoofdrijbaan_voorlopig')
links_hoofdrijbaan_1

True
True
914 914
1581


C:\Users\lucas.demeyer\AppData\Roaming\Python\Python311\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,NO,FROMNODENO,TONODENO,TYPENO,TSYSSET,NUMLANES,VISUEELL~1,V_WET,geometry,TYPE_1
47,1001,1459,1457,2,"BUS,FIETS,PW,VR",3,1,NaN,"LINESTRING (647888.96 694241.627, 647881.253 6...",hoofdrijbaan_voorlopig
48,1002,1465,1462,1,"BUS,FIETS,PW,VR",2,1,NaN,"LINESTRING (647962.667 694487.472, 647949.317 ...",hoofdrijbaan_voorlopig
50,1004,1476,1475,1,"BUS,FIETS,PW,VR",2,1,NaN,"LINESTRING (648424.904 696004.592, 648401.036 ...",hoofdrijbaan_voorlopig
51,1005,1463,1464,1,"BUS,FIETS,PW,VR",3,1,NaN,"LINESTRING (647957.725 694422.755, 647959.815 ...",hoofdrijbaan_voorlopig
52,1006,1461,1463,2,"BUS,FIETS,PW,VR",3,1,NaN,"LINESTRING (647933.168 694348.226, 647947.201 ...",hoofdrijbaan_voorlopig
...,...,...,...,...,...,...,...,...,...,...
14058,119654,92778,93406,1,"BUS,FIETS,PW,VR",2,1,NaN,"LINESTRING (636850.937 714193.616, 635934.14 7...",hoofdrijbaan_voorlopig
14060,119666,86083,93418,2,"BUS,FIETS,PW,VR",3,1,NaN,"LINESTRING (655241.013 696277.093, 655240.423 ...",hoofdrijbaan_voorlopig
14061,119689,92798,93440,1,"BUS,FIETS,PW,VR",2,1,NaN,"LINESTRING (672009.824 732071.571, 672047.359 ...",hoofdrijbaan_voorlopig
14065,119702,92789,93452,1,"BUS,FIETS,PW,VR",2,1,NaN,"LINESTRING (673985.783 714923.558, 674074.719 ...",hoofdrijbaan_voorlopig


In [13]:
# Eerste export snelwegen
if export:
    links_hoofdrijbaan_1.to_file(r'E:\v423_ONTW_SPM\04 Diverse onderzoeken\34 Gesloten wegvakprobleem\Code\Output\shp\test_snelwegen.shp')

### 1e selectie op/afit uit knopen met 1 ink & 1 uitg link

In [14]:
eerste_selectie_knopen_op_afrit = selectie_knopen_obv_vistype(nodes_1in_1out, vis_in=['3', '4'], vis_out=['3', '4'])
eerste_selectie_knopen_op_afrit

,NO,CODE,L_IN_NO,L_IN_F_NO,L_IN_T_NO,L_IN_VIS,L_IN_VWET,L_OUT_NO,L_OUT_F_NO,L_OUT_T_NO,L_OUT_VIS,L_OUT_VWET,STUDIEGE~3
2,6,None,2562,74,6,4,None,2047,6,810,4,None,1.0
3,9,None,446,358,9,4,None,796,9,360,4,None,1.0
9,53,None,2009,1445,53,4,None,1945,53,230,4,None,1.0
10,66,None,958,793,66,4,None,958,66,793,4,None,1.0
14,73,None,41,803,73,4,None,2046,73,802,4,None,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11004,92847,None,115784,85507,92847,4,None,115903,92847,85372,4,None,1.0
11015,93200,None,119298,85367,93200,3,None,119299,93200,85263,3,None,1.0
11016,93201,None,119300,85479,93201,3,None,119301,93201,85506,3,None,1.0
11019,93253,None,119398,93251,93253,4,None,119399,93253,85250,4,None,1.0


In [15]:
links_op_af_rit_1 = filter_links_obv_knopen(eerste_selectie_knopen_op_afrit, links, link_type_var='TYPE_1', link_type_value='op-/afrit_voorlopig')
links_op_af_rit_1

True
True
1016 1016
1798


C:\Users\lucas.demeyer\AppData\Roaming\Python\Python311\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,NO,FROMNODENO,TONODENO,TYPENO,TSYSSET,NUMLANES,VISUEELL~1,V_WET,geometry,TYPE_1
0,2,302,805,11,"BUS,FIETS,PW,VR",4,4,NaN,"LINESTRING (656084.017 693089.769, 656066.497 ...",op-/afrit_voorlopig
1,4,450,308,11,"BUS,FIETS,PW,VR",1,4,NaN,"LINESTRING (656906.921 688040.541, 656888.69 6...",op-/afrit_voorlopig
7,28,805,301,10,"BUS,FIETS,PW,VR",2,3,NaN,"LINESTRING (656005.535 693030.681, 655986.659 ...",op-/afrit_voorlopig
11,41,803,73,12,"BUS,FIETS,PW,VR",1,4,NaN,"LINESTRING (656071.516 692434.468, 656095.381 ...",op-/afrit_voorlopig
14,44,664,663,11,"BUS,FIETS,PW,VR",3,4,NaN,"LINESTRING (647745.345 694555.199, 647732.023 ...",op-/afrit_voorlopig
...,...,...,...,...,...,...,...,...,...,...
14025,119398,93251,93253,6,"BUS,FIETS,PW,TRA,TVOV,VR",1,4,NaN,"LINESTRING (647956.518 711155.523, 647939.078 ...",op-/afrit_voorlopig
14026,119399,93253,85250,6,"BUS,FIETS,PW,TRA,TVOV,VR",1,4,NaN,"LINESTRING (647875.57 711103.25, 647844.214 71...",op-/afrit_voorlopig
14036,119429,86987,93268,11,"BUS,FIETS,PW,VR",2,4,NaN,"LINESTRING (655191.224 713299.187, 655175.619 ...",op-/afrit_voorlopig
14037,119430,93268,89540,11,"BUS,FIETS,PW,VR",1,4,NaN,"LINESTRING (655165.119 713239.413, 655144.222 ...",op-/afrit_voorlopig


In [16]:
# Eerste export op-/afritten
if export:
    links_op_af_rit_1.to_file(r'E:\v423_ONTW_SPM\04 Diverse onderzoeken\34 Gesloten wegvakprobleem\Code\Output\shp\test_op_afritten.shp')

### testjes

In [ ]:
links[links['NO'] == 1943]

In [ ]:
links[links['NO'] == 2509]

In [ ]:
x = nodes[~nodes['L_IN_NO'].str.contains(',', na=False)]
x

In [ ]:
x[x['L_OUT_NO'].str.contains(',', na=False)]


In [ ]:
x[x['L_IN_VIS'] == x['L_OUT_VIS']]

In [ ]:
y = nodes[nodes['L_OUT_NO'].str.contains(',', na=False)]
y

In [ ]:
links[links['NO'] == 3201]

In [ ]:
nodes_1in_1out[nodes_1in_1out['L_IN_VIS'] != nodes_1in_1out['L_OUT_VIS']]

In [ ]:
x = nodes_1in_1out[
    (nodes_1in_1out['L_IN_VIS'].notna()) &
    (nodes_1in_1out['L_OUT_VIS'].notna()) &
    (nodes_1in_1out['L_IN_VIS'] != nodes_1in_1out['L_OUT_VIS'])]
x

In [ ]:
drie_of_meer_links_nodes = nodes[
    (nodes['L_IN_NO'].str.count(',').fillna(0) + 1 >= 3) |
    (nodes['L_OUT_NO'].str.count(',').fillna(0) + 1 >= 3)
]

In [ ]:
nodes_multipe_in_or_out[
    nodes_multipe_in_or_out['L_IN_VIS'].isin(['1', '2']) &
    nodes_multipe_in_or_out['L_OUT_VIS'].isin(['1', '2'])
]

In [18]:
#vereenvoudigen op- en afritten --> enkel links (knopen) behouden die rechstreeks aansluiten op snelweg

# --> later

"""
nodes_multipe_in_or_out[
    nodes_multipe_in_or_out['L_IN_VIS'].isin(['3', '4']) &
    nodes_multipe_in_or_out['L_OUT_VIS'].isin(['3', '4'])
]
"""


"\nnodes_multipe_in_or_out[\n    nodes_multipe_in_or_out['L_IN_VIS'].isin(['3', '4']) &\n    nodes_multipe_in_or_out['L_OUT_VIS'].isin(['3', '4'])\n]\n"